In [13]:
import uproot
import matplotlib
import matplotlib.pyplot as plt
import awkward as ak
import hist
import numpy as np
from event_display import gdml_read_ecal_info
import os

%matplotlib inline
matplotlib.rc("figure", dpi=200)

dir_path = os.path.dirname(os.path.realpath("__file__"))
print(f"Believed path is: {dir_path}")

Believed path is: /home/romanov/eic/eic_ai_pid/data/simulation


In [14]:
# Data file name
#data_file_name = f"{dir_path}/data/test_gun.edm4hep.root"
file_with_event_num = 100000
root_file_name = f"{dir_path}/../disk_gun_electrons_0-15GeV_{file_with_event_num}ev.edm4hep.root"
modules_file_name = f'{dir_path}/../disk_gun_electrons_0-15GeV_{file_with_event_num}ev.modules.csv'
energy_file_name = f'{dir_path}/../disk_gun_electrons_0-15GeV_{file_with_event_num}ev.energy.csv'
position_file_name = f'{dir_path}/../disk_gun_electrons_0-15GeV_{file_with_event_num}ev.position.csv'

In [15]:
# Open root file and get "events" tree from it
tree = uproot.open(root_file_name)["events"]

# Number of events to process:
entry_start = 0                  # (!) Try changing those
entry_stop = file_with_event_num                 # <==

gen_status = tree['MCParticles/MCParticles.generatorStatus'].array(entry_start=entry_start, entry_stop=entry_stop)

stable_only = gen_status > 0
# stable_only = stable_only[stable_only]

times = tree['MCParticles/MCParticles.time'].array(entry_start=entry_start, entry_stop=entry_stop)
masses = tree['MCParticles/MCParticles.mass'].array(entry_start=entry_start, entry_stop=entry_stop)
print(stable_only[:3])
print(times[:3])
print(masses[:3])

px = tree['MCParticles/MCParticles.momentum.x'].array(entry_start=entry_start, entry_stop=entry_stop)
py = tree['MCParticles/MCParticles.momentum.y'].array(entry_start=entry_start, entry_stop=entry_stop)
pz = tree['MCParticles/MCParticles.momentum.z'].array(entry_start=entry_start, entry_stop=entry_stop)
pdg = tree['MCParticles/MCParticles.PDG'].array(entry_start=entry_start, entry_stop=entry_stop)

masses = masses[stable_only]
px = px[stable_only]
py = py[stable_only]
pz = pz[stable_only]

e = np.sqrt(masses*masses + px*px + py*py + pz*pz)
print(e[:3])

e = ak.flatten(e).to_numpy()
print(e[:3])
# Read energies, x and y positions
# Flatten arrays for simplicity
#energies = get_flatten_branch_data('EcalEndcapNHits/EcalEndcapNHits.energy')
#hits_x = get_flatten_branch_data('EcalEndcapNHits/EcalEndcapNHits.position.x')
#hits_y = get_flatten_branch_data('EcalEndcapNHits/EcalEndcapNHits.position.y')

[[True, False, False], [True, False], [True]]
[[0, 2.26, 6.43], [0, 3.62], [0]]
[[0.000511, 0.000511, 0.000511], [0.000511, 0.000511], [0.000511]]
[[12.4], [0.843], [1.86]]
[12.44688239  0.84302417  1.86409661]


In [16]:
np.savetxt(energy_file_name, e, comments="True energy of events [GeV]")

In [17]:
pos_x = tree['MCParticles/MCParticles.vertex.x'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_y = tree['MCParticles/MCParticles.vertex.y'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_z = tree['MCParticles/MCParticles.vertex.z'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_x = ak.flatten(pos_x[stable_only]).to_numpy()
pos_y = ak.flatten(pos_y[stable_only]).to_numpy()
pos_z = ak.flatten(pos_z[stable_only]).to_numpy()

pos = np.column_stack((pos_x, pos_y, pos_z))
np.shape(pos)
np.savetxt(position_file_name, e, comments="True position of particle vertex [mm] (direction is 0,0,-1)")

In [18]:
position_file_name

'/home/romanov/eic/eic_ai_pid/data/simulation/../disk_gun_electrons_0-15GeV_100000ev.position.csv'

In [19]:
# Load geometry file
ecal_info = gdml_read_ecal_info(f"{dir_path}/../ecce.gdml")

# ecal_info is of EcalGeoInfo class, which is a helper holding information
# about all needed ecal geometries.
# Print what information it holds:
ecal_info.print()

module_size_x    : 20.5
module_size_y    : 20.5
module_size_z    : 200.0
total_modules    : 2816
num_modules_x    : 61
num_modules_y    : 61
min_x            : -615.0
max_x            : 615.0
min_y            : -615.0
max_y            : 615.0
min_z            : 0.0
max_z            : 0.0
border_left      : -625.25
border_right     : 625.25
border_top       : 625.25
border_bottom    : -625.25
unit             : mm


In [20]:
events = ecal_info.read_events_from_file(root_file_name, 0, file_with_event_num)

In [21]:
original_shape = np.shape(events)
print(f"Original shape {original_shape}")

flatten_shape = (original_shape[0], original_shape[1] * original_shape[2])
print(f"Flatten shape {flatten_shape}")
reshaped_events = np.reshape(events, flatten_shape)

Original shape (100000, 61, 61)
Flatten shape (100000, 3721)


In [22]:
np.savetxt(modules_file_name, reshaped_events)